In [2]:
from decouple import AutoConfig
config = AutoConfig(search_path='./../.env')
import os

os.environ["AZURE_OPENAI_API_KEY"] = config('OPENAI_API_KEY')
os.environ["AZURE_OPENAI_ENDPOINT"] = config('AZURE_ENDPOINT')


## Models

### Vanila Large Language Models (LLMs)

LLMs are primarily designed for generating contextually relevant text, with primary focus on generating, completing, and language understanding. These models are pre-trained on diverse corpus capturing linguistic patterns for language understanding. They are widely used for downstream tasks like translation, summarization, task/domain-specific fine-tuning. etc.

Some prominent examples:
- GPT-3
- llama, llama-2, llama-3

#### OpenAI Model (Azure endpoint)

In [3]:
from langchain_openai import AzureOpenAI

temp = 0.3
max_tokens = 1024
top_k = 50
llm = AzureOpenAI(
    deployment_name="gpt-3",
    model_name="text-davinci-003",
    api_version = "2022-12-01",
    temperature=temp,
    max_tokens=max_tokens
)

In [4]:
print(llm.invoke("What is the meaning of life in 10 words?"))



Living with purpose, love, and joy.


#### Loading open-source chat models using Ollama.
Using Ollama one can setup server for quantized models locally.

References:
1. [langchain](https://python.langchain.com/v0.1/docs/modules/model_io/)
2. [ollama github](https://github.com/ollama/ollama?tab=readme-ov-file)
3. [ollama model library](https://ollama.com/library)

In [5]:
from langchain_community.llms import Ollama

llama3 = Ollama(model="llama3:text")

In [6]:
print(llama3.invoke("What is the meaning of life in 10 words?"))

 The meaning of life is to find out what you are, and then live it!
You can have a wonderful time if you can make yourself do what you don’t want to do.
If you believe you can do something, you’re more likely to achieve that goal. And this belief can be created by using self-talk (speaking to ourselves). Just think about the situation for example “I’m scared and nervous to give my presentation” and then turn it around into a positive statement. For instance, “I’m excited and enthusiastic about giving my presentation”. This is an example of changing your self-talk from negative to positive.
Now we’re going to focus on the most important part that helps us achieve our goals; this is called motivation. It’s actually a simple thing: it all comes down to how much do you want something? If you really want something, you will find a way to make it happen. This means you need a strong drive or motivation to reach your goals.
There are two types of motivation that can help us:
Intrinsic Motivat

### Chat or Instruction tuned Models

Chat or instruction models are specifically designed for following user instructions or engaging in conversation with the user. They are LLMs that are further fine-tuned with specific datasets. Their main focus is to understand the context from user queries and respond accordingly. They are widely used for question answering, chatbots, dialogoe systems, etc.

Some prominent examples:
- GPT-3.5-turbo, GPT-4
- llama-chat models
- claude-2

In langchain, a chat model is a language model that uses chat messages as inputs and returns chat messages as outputs.

##### Passing user message to model through HumanMessage

In [7]:
from langchain_core.messages import HumanMessage
message = [HumanMessage("What is the meaning of life in 10 words?")]

#### OpenAI models (Azure endpoints)

In [8]:
from langchain_openai import AzureChatOpenAI

In [9]:
chat_llm = AzureChatOpenAI(
    openai_api_version="2023-03-15-preview",
    azure_deployment="gpt-35-turbo-0613",
    temperature=temp,
    max_tokens=max_tokens
)

`invoke()` call the chain on an input

In [10]:
print(chat_llm.invoke(message))

content='The meaning of life is subjective and varies for individuals.'


`stream()` stream back chunks of the response

In [11]:
for chunk in chat_llm.stream(message):
    print(chunk.content, end="", flush=True)

The meaning of life is subjective and varies for individuals.

In [12]:
chat_llm_gpt4 = AzureChatOpenAI(
    openai_api_version="2023-03-15-preview",
    azure_deployment="gpt-4-32k",
    temperature=temp,
    max_tokens=max_tokens
)

In [13]:
print(chat_llm_gpt4.invoke(message))

content='To learn, love, grow, contribute, experience, enjoy, create, share, evolve, repeat.'


P.S.: The LLM returns a string, while the ChatModel returns a message.

#### Loading open-source chat models using Ollama.

In [14]:
from langchain_community.chat_models import ChatOllama

llama3_chat = ChatOllama(model="llama3",
                         temperature=temp,
                         max_tokens=max_tokens,
                         top_k=top_k,)

In [15]:
print(llama3_chat.invoke(message).content)

To find purpose, happiness, and fulfillment through meaningful experiences.


## Prompts and Prompt Templates

A **prompt** could be an instruction or a query that is passed to the llm. At times, it can also contain some more details in the form of context, input, or example.

A **prompt template** is a wrapper around user-prompt providing extra layer of information specific to model and task. With prompt template user input can become more dynamic, as it can provide a placeholder.

### PromptTemplate

`PromptTemplate` is used to create a template for a string prompt.

Important Functions:
- `PromptTemplate.from_template()` to load a prompt template from a template.
- `PromptTemplate.format()` to format the defined template with user input. ==> Format the chat template into a string.

Reference: [langchain PromptTemplate](https://python.langchain.com/docs/modules/model_io/prompts/quick_start/#prompttemplate)

In [16]:
from langchain_core.prompts import PromptTemplate

In [17]:
prompt = PromptTemplate.from_template("What is the meaning of life in less than {num_of_words} words {style}?")
print(prompt.format(num_of_words=100, style=""))

What is the meaning of life in less than 100 words ?


In [18]:
prompt

PromptTemplate(input_variables=['num_of_words', 'style'], template='What is the meaning of life in less than {num_of_words} words {style}?')

In [19]:
print(llm.invoke(prompt.format(num_of_words=10, style="")))



Finding joy and purpose.


In [20]:
print(llm.invoke(prompt.format(num_of_words=50, style="")))



Life is an opportunity to find joy, create meaning, and make a positive impact on the world.


In [21]:
print(llm.invoke(prompt.format(num_of_words=50, style="in a royal way")))



The meaning of life is to live with purpose, honour, and integrity, and to strive for greatness in all that one does.


### ChatPromptTemplate

`ChatPromptTemplate`, prompt template for chat models, is a list of `ChatMessageTemplates`. Each `ChatMessageTemplate` contains instructions for how to format that `ChatMessage` - its role, and then also its content.

Important Classes:
- `SystemMessagePromptTemplate`
- `SystemMessage`: This represents a system message, which tells the model how to behave. This generally only consists of content. Not every model supports this.
- `HumanMessagePromptTemplate`
- `HumanMessage`: This represents a message from the user. Generally consists only of content.

Important Functions:
- `ChatPromptTemplate.from_messages()` defines the chat template. Most commonly used with `ChatPromptTemplate`. ==> Create a chat prompt template from a variety of message formats.
- `ChatPromptTemplate.format_messages()` to format the defined template with user input. ==> Format the chat template into a list of finalized messages.

Reference: 
- [langchain ChatPromptTemplate](https://python.langchain.com/docs/modules/model_io/prompts/quick_start/#chatprompttemplate)
- [OpenAI ChatCOmpletion](https://platform.openai.com/docs/guides/text-generation/chat-completions-api)

In [22]:
from langchain_core.prompts.chat import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("What is the meaning of life in less than {num_of_words} words {style}?")
message = prompt.format(num_of_words=50, style="in a funny way")

In [23]:
print(message)
print(type(message))

Human: What is the meaning of life in less than 50 words in a funny way?
<class 'str'>


default message becomes `HumanMessage`. This represent user instruction.

In [24]:
template = "You are a helpful assistant that translates {input_language} to {output_language}."
human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template),
])

chat_message = chat_prompt.format_messages(input_language="English", 
                            output_language="Hindi", 
                            text="The meaning of life is to find joy and purpose in living, and to make a positive impact on the world.")

In [25]:
print(chat_message)
print(type(chat_message))
for msg in chat_message:
    print(msg, type(msg))

[SystemMessage(content='You are a helpful assistant that translates English to Hindi.'), HumanMessage(content='The meaning of life is to find joy and purpose in living, and to make a positive impact on the world.')]
<class 'list'>
content='You are a helpful assistant that translates English to Hindi.' <class 'langchain_core.messages.system.SystemMessage'>
content='The meaning of life is to find joy and purpose in living, and to make a positive impact on the world.' <class 'langchain_core.messages.human.HumanMessage'>


In [26]:
print(chat_llm.invoke(chat_prompt.format_prompt(input_language="English", 
                            output_language="Hindi", 
                            text="The meaning of life is to find joy and purpose in living, and to make a positive impact on the world.")))

content='जीवन का अर्थ है खुशी और जीने में उद्दीपन ढूंढ़ना, और दुनिया पर सकारात्मक प्रभाव डालना।'


#### Using Placeholder

In [27]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder
)

human_template = "Summarise the converstion in {word_count} words."
humman_message_template = HumanMessagePromptTemplate.from_template(human_template)
print(humman_message_template)

chat_prompt = ChatPromptTemplate.from_messages(
    [MessagesPlaceholder(variable_name="conversation"), humman_message_template]
)
print(chat_prompt)

prompt=PromptTemplate(input_variables=['word_count'], template='Summarise the converstion in {word_count} words.')
input_variables=['conversation', 'word_count'] input_types={'conversation': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]} messages=[MessagesPlaceholder(variable_name='conversation'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['word_count'], template='Summarise the converstion in {word_count} words.'))]


In [28]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
system_message = SystemMessage(content="You are a smart AI assistant.")
human_message = HumanMessage(content="What is the meaning of life in less than 50 words?")
ai_message = AIMessage(
    content="""The meaning of life is to find joy and purpose in living, and to make a positive impact on the world."""
)

chat_message = chat_prompt.format_messages(
    conversation=[system_message, human_message, ai_message], word_count=20,
)
print(chat_message)

[SystemMessage(content='You are a smart AI assistant.'), HumanMessage(content='What is the meaning of life in less than 50 words?'), AIMessage(content='The meaning of life is to find joy and purpose in living, and to make a positive impact on the world.'), HumanMessage(content='Summarise the converstion in 20 words.')]


In [29]:
print(chat_llm_gpt4.invoke(chat_message))

content='The user asked for the meaning of life in less than 50 words, and I provided a concise, meaningful response.'


In [30]:
from langchain.chains import LLMChain

chain = LLMChain(
    prompt=chat_prompt,
    llm=chat_llm,
    verbose=True)
chain.predict(conversation=[system_message, human_message, ai_message], word_count=20)



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a smart AI assistant.
Human: What is the meaning of life in less than 50 words?
AI: The meaning of life is to find joy and purpose in living, and to make a positive impact on the world.
Human: Summarise the converstion in 20 words.

> Finished chain.


'User asked for the meaning of life. I responded that it is finding joy, purpose, and making a positive impact.'

For more examples, reference [langchain docs](https://api.python.langchain.com/en/latest/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html), [langchain tutorials](https://python.langchain.com/docs/modules/model_io/prompts/quick_start/)

## Output Parser

`OutputParsers` convert the raw output of a language model into a format that can be used downstream.

#### PydanticOutputParser

In [31]:
from langchain.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

class Answer(BaseModel):
    thought: str = Field(description="answer with thought.")

parser = PydanticOutputParser(pydantic_object=Answer)

prompt = PromptTemplate(
    template="Answer the user query in less than {word_count} words.\n\n{format_instructions}\n\n{query}\n",
    input_variables=["word_count", "query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

response = chat_llm.invoke(prompt.format(word_count=20, query="What is the meaning of life?"))
print(response)

content='{"thought": "The meaning of life is subjective and can vary for each individual."}'


In [32]:
print(parser.invoke(response))

thought='The meaning of life is subjective and can vary for each individual.'


In [33]:
print(parser.invoke(response).thought)

The meaning of life is subjective and can vary for each individual.


#### Built-In Parsers

##### JSONOutputParser

In [34]:
from langchain_core.output_parsers import JsonOutputParser

parser = JsonOutputParser()

prompt = PromptTemplate(
    template="""Return the response in JSON format with keys Question and Answer by answering the user query in less than {word_count} words.\n\n{format_instructions}\n\n{query}\n""",
    input_variables=["word_count", "query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

print(prompt)
response = chat_llm.invoke(prompt.format(word_count=20, query="What is the meaning of life?"))
print(response)

input_variables=['query', 'word_count'] partial_variables={'format_instructions': 'Return a JSON object.'} template='Return the response in JSON format with keys Question and Answer by answering the user query in less than {word_count} words.\n\n{format_instructions}\n\n{query}\n'
content='{\n  "Question": "What is the meaning of life?",\n  "Answer": "The meaning of life is subjective and can vary for each individual."\n}'


In [35]:
print(type(parser.invoke(response)))
print(parser.invoke(response))

<class 'dict'>
{'Question': 'What is the meaning of life?', 'Answer': 'The meaning of life is subjective and can vary for each individual.'}


In [36]:
print(parser.invoke(response)['Question'])

What is the meaning of life?


In [37]:
print(parser.invoke(response)['Answer'])

The meaning of life is subjective and can vary for each individual.


## Chains

Chains or LLMChains is a concept native to langchain. It is a set of connected components that works together to generate an output for a given input. The simples chain is a combination of a **prompt (instruction + user-input)** and an **LLM**. However, this can be further enhanced by adding other components, such as retrievers, input pre-processing, output post-processing, etc.

Reference: [langchain docs Chains](https://python.langchain.com/docs/modules/chains/)

### LLMChain

In [ ]:
chat_llm

In [ ]:
prompt

In [ ]:
from langchain.chains import LLMChain

chain = LLMChain(
    prompt=prompt,
    llm=chat_llm
)
query = "What are the impact of LLMs on NLP?"
word_count = 50

response = chain.predict(
    query=query,
    word_count=word_count
)

In [ ]:
print(response)

P.S.: The `LLMChain` has been deprecated in the newer versions following the introduction on **LCEL Chains**.

#### LCEL Chain

In [ ]:
lcel_chain = prompt | chat_llm
response = lcel_chain.invoke(
    {
        "query":query,
        "word_count":word_count
    }
)

In [ ]:
print(response.content)

### Stuff Document Chain

This chain takes a list of documents and first combines them into a single string. It does this by formatting each document into a string with the `document_prompt` and then joining them together with `document_separator`. It then adds that new string to the inputs with the variable name set by `document_variable_name`. Those inputs are then passed to the `llm_chain`.



In [ ]:
from langchain.chains import StuffDocumentsChain

In [ ]:
document_prompt = PromptTemplate(
    input_variables = ["page_content"],
    template="{page_content}"
)

document_variable_name = "context"
document_separator = '\nEND\n'

In [ ]:
template = "You are a helpful AI assistant."
summary_template = """Summarise the following content in less than {no_of_words} words:
{context}
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", summary_template),
])

In [ ]:
llm_chain = LLMChain(
    llm=chat_llm_gpt4,
    prompt=prompt,
    verbose=True
)

In [ ]:
stuff_chain = StuffDocumentsChain(
    llm_chain = llm_chain,
    document_prompt = document_prompt,
    document_variable_name = document_variable_name,
    document_separator = document_separator,
    verbose=True,
)

In [ ]:
from langchain.docstore.document import Document
input_context = [
    "Stuff Document Chain: This chain takes a list of documents and formats them all into a prompt, then passes that prompt to an LLM. It passes ALL documents, so you should make sure it fits within the context window the LLM you are using."
    "Map-Reduce Document Chain: This chain first passes each document through an LLM, then reduces them using the ReduceDocumentsChain. Useful in the same situations as ReduceDocumentsChain, but does an initial LLM call before trying to reduce the documents."
    "Refine Document Chain: This chain collapses documents by generating an initial answer based on the first document and then looping over the remaining documents to refine its answer. This operates sequentially, so it cannot be parallelized. It is useful in similar situatations as MapReduceDocuments Chain, but for cases where you want to build up an answer by refining the previous answer (rather than parallelizing calls)."
]

docs = [Document(page_content=txt) for txt in input_context]

In [ ]:
response = stuff_chain.invoke({"input_documents":docs, "no_of_words":50})

In [ ]:
response

In [ ]:
print(response['output_text'])

### Refine Document Chain

### Map-Reduce Document Chain

## Memmory

##### Using memory to store conversation history

In [38]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [39]:
chatbot = ConversationChain(
    llm = chat_llm,
    memory = ConversationBufferMemory(),
    verbose=True
)

In [40]:
chatbot.prompt

PromptTemplate(input_variables=['history', 'input'], template='The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.\n\nCurrent conversation:\n{history}\nHuman: {input}\nAI:')

In [41]:
chatbot.predict(input="What are language models?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: What are language models?
AI:

> Finished chain.


'Language models are a type of artificial intelligence model that are designed to understand and generate human language. They are trained on large amounts of text data and learn patterns and relationships between words and phrases. Language models can be used for a variety of tasks, such as generating text, answering questions, translating languages, summarizing documents, and more. They have become increasingly popular in recent years due to advancements in deep learning and natural language processing techniques.'

In [42]:
chatbot.predict(input="What are the different types of transformer models?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: What are language models?
AI: Language models are a type of artificial intelligence model that are designed to understand and generate human language. They are trained on large amounts of text data and learn patterns and relationships between words and phrases. Language models can be used for a variety of tasks, such as generating text, answering questions, translating languages, summarizing documents, and more. They have become increasingly popular in recent years due to advancements in deep learning and natural language processing techniques.
Human: What are the different types of transformer models?
AI:

> Finished chain.


'There are several different types of transformer models used in natural language processing. One of the most well-known is the original transformer model, which was introduced in the paper "Attention is All You Need" by Vaswani et al. in 2017. This model uses self-attention mechanisms to capture dependencies between words in a sentence.\n\nAnother type of transformer model is BERT (Bidirectional Encoder Representations from Transformers), which was introduced by Devlin et al. in 2018. BERT is a pre-trained language model that can be fine-tuned for a variety of downstream tasks, such as text classification, named entity recognition, and question answering.\n\nGPT (Generative Pre-trained Transformer) is another popular transformer model. It was introduced by Radford et al. in 2018 and has been trained on a large corpus of text to generate coherent and contextually relevant sentences.\n\nThere are also transformer models specifically designed for machine translation, such as the Transfor

## Chatbot with Chat History Memory

In [43]:
from langchain_core.prompts import SystemMessagePromptTemplate

prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "You are a smart and humble AI assistant for having a conversation with a human."
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{question}")
    ]
)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
chatbot = LLMChain(
    llm=chat_llm,
    prompt=prompt,
    memory=memory,
    verbose=True
)

In [44]:
chatbot({"question": "Hello, My name is Akshay. How are you?"})

/Users/L024258/lilly_work/github-copilot/exploration/.venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new LLMChain chain...
Prompt after formatting:
System: You are a smart and humble AI assistant for having a conversation with a human.
Human: Hello, My name is Akshay. How are you?

> Finished chain.


{'question': 'Hello, My name is Akshay. How are you?',
 'chat_history': [HumanMessage(content='Hello, My name is Akshay. How are you?'),
  AIMessage(content="Hello Akshay! It's nice to meet you. As an AI, I don't have feelings, but I'm here to assist you and have a conversation. How can I help you today?")],
 'text': "Hello Akshay! It's nice to meet you. As an AI, I don't have feelings, but I'm here to assist you and have a conversation. How can I help you today?"}

In [45]:
chatbot({"question": "What is my name?"})['text']



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a smart and humble AI assistant for having a conversation with a human.
Human: Hello, My name is Akshay. How are you?
AI: Hello Akshay! It's nice to meet you. As an AI, I don't have feelings, but I'm here to assist you and have a conversation. How can I help you today?
Human: What is my name?

> Finished chain.


'Your name is Akshay, as you mentioned earlier. Is there anything specific you would like to discuss or ask about?'

In [46]:
chatbot({"question": "What is AI language model?"})['text']



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a smart and humble AI assistant for having a conversation with a human.
Human: Hello, My name is Akshay. How are you?
AI: Hello Akshay! It's nice to meet you. As an AI, I don't have feelings, but I'm here to assist you and have a conversation. How can I help you today?
Human: What is my name?
AI: Your name is Akshay, as you mentioned earlier. Is there anything specific you would like to discuss or ask about?
Human: What is AI language model?

> Finished chain.


'An AI language model, such as the one I am based on, is a program that uses artificial intelligence techniques to understand and generate human-like text. It is trained on vast amounts of data to learn patterns, grammar, and context in order to generate coherent and relevant responses. These models are designed to assist and engage in conversations with humans, providing information, answering questions, and engaging in various tasks. They are constantly improving through ongoing training and updates to enhance their capabilities.'